In [1]:
import netket as nk
import json
from qutip import *
import numpy as np
import time
import multiprocessing as mp
from collections import OrderedDict
from pickle import dump
import os
import matplotlib.pyplot as plt
import scipy
from matplotlib import gridspec
from functools import reduce
plt.style.use('seaborn')
from scipy.stats import norm
import sys
sys.path.append("/Users/victorwei/Research projects/Neural Network Quantum State/penalty excited states")
import expect_grad_ex
import vmc_ex
import jax
import optax
from mpi4py import MPI
import mpi4jax
#comm = MPI.COMM_WORLD
#rank = comm.Get_rank()
#print(rank)

In [7]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def exactDiagonalization(hamiltonian):
    # Changes Hamiltonian to matrix form, where hamiltonian of interest is sparse in matrix form
    #haMatrix = hamiltonian.to_sparse()
    # Gets eigenvalues and vectors, where the built-in function uses 
    eigenValues, v = nk.exact.lanczos_ed(hamiltonian, compute_eigenvectors=True)

    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

def exactDiagonalization_full(hamiltonian):
    # Changes Hamiltonian to matrix form
    haMatrix = hamiltonian.to_dense()
    # Gets eigenvalues and vectors
    eigenValues, v = np.linalg.eigh(haMatrix)
    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

# NetKet RBM with stochastic reconfiguration descent
class RBM:
    def __init__(self, N, hamiltonian, hilbertSpace, machine):
        # Assign inputsv[:, i]
        self.hamiltonian, self.hilbertSpace, self.machine, self.N = hamiltonian, hilbertSpace, machine, N
        # Define sampler
        self.sampler = nk.sampler.MetropolisLocal(hilbert=hilbertSpace)
        # Define optimizer
        self.optimizer = nk.optimizer.Sgd(learning_rate=0.01)
        # Define Stochastic reconfiguration
        self.sr = nk.optimizer.SR(diag_shift=0.01) #diagnol shift, its role as regularizer? seems to take a different form as 
        #compared to the version I have seen
        # Variational state
        self.vs = nk.vqs.MCState(self.sampler, self.machine, n_samples=3000, n_discard_per_chain=300) #discarded number of samples 
        #at the beginning of the MC chain

    # Output is the name of the output file in which the descent data is stored
    def __call__(self, output, state_list, shift_list):
        self.vs.init_parameters(jax.nn.initializers.normal(stddev=0.25))
        gs = vmc_ex.VMC_ex(hamiltonian=self.hamiltonian, optimizer=self.optimizer, variational_state=self.vs, preconditioner=self.sr, 
                   state_list = state_list, shift_list = shift_list)
        # Start timing
        start = time.time()
        # Set the output files as well as number of iterations in the descent
        gs.run(out=output, n_iter=100)
        end = time.time()
        runTime = end - start
        # Import the data from log file
        data = json.load(open(output + '.log'))
        # Extract the relevant information
        # iters = data["Energy"]["iters"]
        energy_RBM = data["Energy"]["Mean"]["real"] #get the real part of the mean energy
       
        # finalEng = energy_RBM[-1]
        finalEng = reduce(lambda x, y: x if y is None else y, energy_RBM)
        # Get machine statethe state of the machine as an array
        state = self.vs.to_array()
        # Outputs the final energy, the final state, and the runtime
        parameters = self.vs.parameters
        # Outputs the final energy, the final state, and the runtime
        return finalEng, state, self.vs
    
# Error Calculation (Input: the found state, the state from exact diagonalization, the found energy, the energy from exact diagonalization)
def err(state, edState, eng, edEng,N):
    engErr = np.abs(eng - edEng)
    overlap = np.dot(state.conj().reshape(2**N, 1).T, edState.reshape(2**N, 1))
    waveFunctionErr = 1 - (np.linalg.norm(overlap))**2
    return engErr, waveFunctionErr


# Combines all steps into a function to run on the cluster
def runDescentCS(N,B,Ak,alpha, state_list, shift_list):
    # Define hamiltonian and hibert space (need to do this here cause can't use netket objects as input to use multiprocessing functions)
    ha, hi = CSHam(N,B,Ak)
    # RBM Spin Machine
    ma = nk.models.RBM(alpha=alpha, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
    # Initialize RBM
    rbm = RBM(N, ha, hi, ma) #an instance of class RBM
    # Run RBM
    eng, state, vstate = rbm("2021_summer_data/ex1_var_N_10_alpha3_1_Logs_sta_er_test"+str(N), state_list, shift_list) #where _call_ will be invoked
    return eng, state, vstate

In [8]:
N = 4
Ak = []

alpha = 3  #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
A = N/2
N0 = N/2
for i in range(N-1):
    # Constant A
    #Ak_i = 1
    # Variable A
    Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
    
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)


#Exact Diagonalization¢
#e, v = exactDiagonalization_full(ha)

ma = nk.models.RBM(alpha=3, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
sampler = nk.sampler.MetropolisLocal(hilbert=hi)
vs1 = nk.vqs.MCState(sampler, ma, n_samples=3000, n_discard_per_chain=300)
vs1.init_parameters(jax.nn.initializers.normal(stddev=0.25))
state_list = [vs1]
shift_list = [0]

eng, state, gs = runDescentCS(N,B,Ak,alpha, state_list, shift_list)

100%|█| 100/100 [00:08<00:00, 11.26it/s, Energy=-0.701-0.002j ± 0.011 [σ²=0.232,


In [103]:
#keep running for a while
optimizer = optax.sgd(learning_rate=0.005)
# Define Stochastic reconfiguration
sr = nk.optimizer.SR(diag_shift=0.001)

gta = vmc_ex.VMC_ex(hamiltonian=ha, optimizer=optimizer, variational_state=gs, preconditioner=sr, 
                   state_list = state_list, shift_list = shift_list)
output = "2021_summer_data/var_N_10_alpha3_4_Logs"+str(N)
gta.run(out=output, n_iter=300)

100%|█| 300/300 [02:06<00:00,  2.37it/s, Energy=-1.86310+0.00014j ± 0.00011 [σ²=


(JsonLog('2021_summer_data/var_N_10_alpha3_4_Logs10', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.1296858787536621
   	Params: 0.0020046234130859375,)

In [107]:
gs.n_samples = 80000
gs.expect(ha)


-1.86235-0.00003j ± 0.00028 [σ²=0.00346, R̂=1.0001]

In [121]:
#this shit saves the ground state parameters
import pickle
fileName = "2021_summer_data/0.063%_N10_error_gs.json"
file = open(fileName,'wb')
pickle.dump(gs.parameters, file)

In [104]:
c = gs.to_array()


In [4]:
e, v = exactDiagonalization_full(ha)

NameError: name 'ha' is not defined

In [139]:
overlap = np.dot(c.conj().reshape(2**N, 1).T, v[0].reshape(2**N, 1))
Err = 1 - (np.linalg.norm(overlap))**2
print(Err)

0.0006298737753793748


In [120]:
print(v[0])


[0.        +0.j 0.02871584+0.j 0.03530649+0.j ... 0.        +0.j
 0.        +0.j 0.        +0.j]


In [9]:
#this shit loads the ground state parameters
import pickle
fileName = "2021_summer_data/0.063%_N10_error_gs.json"
file = open(fileName,'rb')
a = pickle.load(file)
gs.parameters = a

In [12]:
#we now calculate the excited states
N = 10
Ak = []

alpha = 3  #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
A = N/2
N0 = N/2
for i in range(N-1):
    # Constant A
    #Ak_i = 1
    # Variable A
    Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
    
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)


#Exact Diagonalization¢
e, v = exactDiagonalization_full(ha)




In [14]:
ma = nk.models.RBM(alpha=3, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
sampler = nk.sampler.MetropolisLocal(hilbert=hi)
gs = nk.vqs.MCState(sampler, ma, n_samples=3000, n_discard_per_chain=300)
#this shit loads the ground state parameters
import pickle
fileName = "2021_summer_data/0.063%_N10_error_gs.json"
file = open(fileName,'rb')
a = pickle.load(file)
gs.parameters = a
#gs.init_parameters(jax.nn.initializers.normal(stddev=0.25))
state_list1 = [gs]
shift_list1 = [0.08]

eng1, state1, ex_1 = runDescentCS(N,B,Ak,alpha, state_list1, shift_list1)

/Users/victorwei/opt/miniconda3/lib/python3.8/site-packages/netket/vqs/mc/mc_state/state.py:58: UserWarning: n_samples=3000 (3000 per MPI rank) does not divide n_chains=16, increased to 3008 (3008 per MPI rank)
  warnings.warn(
100%|█| 1000/1000 [09:21<00:00,  1.78it/s, Energy=-1.6945+0.0014j ± 0.0021 [σ²=0


In [37]:
#keep running for a while
optimizer = optax.sgd(learning_rate=0.1)
# Define Stochastic reconfiguration
sr = nk.optimizer.SR(diag_shift=0.01)

#adjust number of samples
ex_1.n_samples = 3000
gs.n_samples = 3000
state_list1 = [gs]
shift_list1 = [0.08]

gta = vmc_ex.VMC_ex(hamiltonian=ha, optimizer=optimizer, variational_state=ex_1, preconditioner=sr, 
                   state_list = state_list1, shift_list = shift_list1)
output = "2021_summer_data/ex1_var_N_10_alpha3_pen0.8_Logs"+str(N)+"_4_sta_er"
gta.run(out=output, n_iter=1000)

100%|█| 1000/1000 [07:05<00:00,  2.35it/s, Energy=-1.77626+0.00057j ± 0.00017 [σ


(JsonLog('2021_summer_data/ex1_var_N_10_alpha3_pen0.8_Logs10_4_sta_er', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.5901055335998535
   	Params: 0.0052301883697509766,)

In [39]:
ex1 = ex_1.to_array()
overlap1 = np.dot(ex1.conj().reshape(2**N, 1).T, v[0].reshape(2**N, 1))
#Err1 = 1 - (np.linalg.norm(overlap1))**2
print(np.linalg.norm(overlap1))

0.032263524574199065


In [11]:
print(gs)

AttributeError: 'MCState' object has no attribute 'dtype'